<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/SMS_Spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import io
tf.__version__

# path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip",
#                                       origin = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",extract=True)


# !unzip $path_to_zip -d data

lines = io.open('data/SMSSpamCollection').read().strip().split('\n')
lines[0]

spam_dataset = []
for line in lines:
  label, text = line.split('\t')
  if label.strip() == 'spam':
    spam_dataset.append((1, text.strip()))
  else:
    spam_dataset.append(((0, text.strip())))
print(spam_dataset[0])



(0, 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')
